## Train a Neural Network

Use PyTorch to train a simple neural network on the data recorded between heuristic MTAG players.

### Convert Recorded Tournaments to Neural Network Data

In [1]:
from pathlib import Path
from os import listdir
from os.path import isfile, join
import struct

training_directory = Path("../../../recorded_games/2024-01-08/mtag/converted")
training_directory.mkdir(parents=True, exist_ok=True)
train_data_directory = Path("../../../recorded_games/2024-01-08/mtag/")

test_directory = Path("../../../recorded_games/2024-01-08/mtag_test_data/converted")
test_directory.mkdir(parents=True, exist_ok=True)
test_data_directory = Path("../../../recorded_games/2024-01-08/mtag_test_data/")

In [2]:
# Create list of all recorded tournaments in directory
train_files = [join(train_data_directory, f) for f in listdir(
        train_data_directory) if isfile(join(train_data_directory, f))]

test_files = [join(test_data_directory, f) for f in listdir(
        test_data_directory) if isfile(join(test_data_directory, f))]

In [3]:
# Initialize and test C shared library
###############################################################################
import ctypes
# Location of C shared library
c_lib = ctypes.CDLL("../../Cpp/lib/lib_deepshark.so")
# %% Set data types of test C function's arguments and return value
add_ints = c_lib.add_ints
add_ints.argtypes = [ctypes.c_int, ctypes.c_int]
add_ints.restype = ctypes.c_int  # C function returns integer
# Test C function
result = add_ints(4, 5)
print(result)

9


In [4]:
# Initalize C function to convert tournament results
###############################################################################
write_nn_vector_data = c_lib.write_nn_vector_data
write_nn_vector_data.argtypes = [ctypes.POINTER(ctypes.c_char), ctypes.POINTER(ctypes.c_char)]
write_nn_vector_data.restype = None

In [5]:
 # Convert recorded tournaments to neural network inputs and labels
def convert_files(data_files, conv_dir):
    for tfile in data_files:
        b_r_fn = tfile.encode("utf-8")  # create byte objects from the strings
        tfilew = str(conv_dir / "converted_file.bin")
        b_w_fn = tfilew.encode("utf-8")
        read_filename = ctypes.c_char_p(b_r_fn)
        write_filename = ctypes.c_char_p(b_w_fn)
        write_nn_vector_data(read_filename, write_filename)

In [6]:
#convert_files(train_files, training_directory)
#convert_files(test_files, test_directory)

### Create custom PyTorch datasets and dataloaders for binary data

In [7]:
import torch
from torch.utils.data import Dataset

class PokerDataset(Dataset):
        def __init__(self, converted_file):
            self.data_file = Path(converted_file)
            self.file_size_in_bytes = self.data_file.stat().st_size
            self.item_size_in_bytes = 8 * 68  # Array of 68 doubles per record
                        
        def __len__(self):
            return int(self.file_size_in_bytes / self.item_size_in_bytes)
        
        def __getitem__(self, idx):
            with open(self.data_file, mode='rb') as f:
                f.seek(idx*self.item_size_in_bytes)
                item = f.read(self.item_size_in_bytes)
            vector = struct.unpack("<"+68*"d", item)
            data = torch.tensor(vector[0:60])
            label = torch.tensor(vector[60:])
            return data, label




In [8]:
from torch.utils.data import DataLoader

batch_size = 64

training_data = PokerDataset(training_directory / "converted_file.bin")
test_data = PokerDataset(test_directory / "converted_file.bin")

train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [9]:
len(train_dataloader.dataset)

325928

In [10]:
train_dataloader.dataset[162964-1]

(tensor([1.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5714,
         0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.2857, 0.0000, 0.0000, 0.0000, 1.0000, 0.2857, 0.0000, 0.0000, 1.0000,
         0.0000, 0.5714, 0.0000, 1.0000, 0.0000, 0.0000, 0.0020, 0.0020, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.1000]),
 tensor([0., 0., 0., 0., 1., 0., 0., 0.]))

### Define Neural Network


In [3]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [7]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(60, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        action_logits = logits[:,:-1]
        ##action_probs = nn.Softmax(dim=1)(action_logits)
        bet_logit = logits[:, -1]
        scaled_bet = nn.Sigmoid()(bet_logit)
        return action_logits, scaled_bet
        #return logits

In [14]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=8, bias=True)
  )
)


In [15]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)
print(pytorch_trainable_params)

6648
6648


In [16]:
X = torch.rand(3, 60, device=device)
output = model(X)
print(output)

(tensor([[-0.2712, -0.1416, -0.2987, -0.0798,  0.1598, -0.2010, -0.1904],
        [-0.2869, -0.1468, -0.2904, -0.0869,  0.1756, -0.2029, -0.1872],
        [-0.2634, -0.1240, -0.3076, -0.0506,  0.1400, -0.1931, -0.2147]],
       grad_fn=<SliceBackward0>), tensor([0.5551, 0.5554, 0.5535], grad_fn=<SigmoidBackward0>))


In [17]:
print(output[0].shape)
print(output[1].shape)

torch.Size([3, 7])
torch.Size([3])


### Train Neural Network

In [18]:
learning_rate = 3e-3
epochs = 100
weights = torch.tensor([5,5,5,5,0.1,5,5])
action_loss_fn = nn.CrossEntropyLoss(weights)
bet_loss_fn = nn.MSELoss()

In [19]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
def loss_fn(pred, y):
    weight = 0.5
    actions_pred = pred[0]
    actions_y = y[:, :-1]
    bet_pred = pred[1]
    bet_y = y[:, -1]
    loss = weight * action_loss_fn(actions_pred, actions_y) + (1. - weight) * bet_loss_fn(bet_pred, bet_y)
    return loss

In [21]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred[0].argmax(1) == y[:, :-1].argmax(1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
model_dir = Path("../../../recorded_games/models")
model_dir.mkdir(parents=True, exist_ok=True)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
    torch.save(model.state_dict(), os.path.join(str(model_dir), 'epoch-{}.pt'.format(t)))
print("Done!")

In [5]:
model = NeuralNetwork()
model.load_state_dict(torch.load("../../../recorded_games/models/epoch-61.pt"))
model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=60, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=8, bias=True)
  )
)

In [8]:
example_input = torch.rand(1, 60, device=device)
model(example_input)

(tensor([[ 14.2165,   1.3539,  -3.2450,   6.0915,   1.8464, -14.3288,  -8.8218]],
        grad_fn=<SliceBackward0>),
 tensor([0.7331], grad_fn=<SigmoidBackward0>))

In [9]:
example_input = torch.rand(1, 60, device=device)
traced_script_module = torch.jit.trace(model, example_input)
traced_script_module.save("traced_poker_model.pt")

In [10]:
3*224*224

150528